In [4]:
#This program is to test the calculations of XBRL data
#I am reading an SQL query to a table, which will then be linked to the taxonomy dictionary to be able to deduce
#the calculations going forward and determine them currently for total labels.

#import all the goodies
import pandas as pd
from pandas import DataFrame as df
import csv
import os
import sqlite3
import numpy as np
import string
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
#change directory and connect to database
os.chdir('F:\Database Project\SQLLITE Studio Files')
con = sqlite3.connect('financial_database.db3')
con.text_factory = lambda x: unicode(x, 'utf-8', 'ignore')

In [5]:
#get a sample SQL query with a company statement of income 
test_fin = pd.read_sql_query('''
select * from num
join pre using ( adsh , tag , version )
where adsh = '0000909832-15-000014'
    and ddate = '20150831'
    and qtrs in (0,4)
    and dimh = '0x00000000'
    and report = 4
order by report, line;
''', con)

In [612]:
#retrieve the table with the calculation definitions from the taxonomy 2017 dictionary
os.chdir('F:\Database Project\Excel Files\Taxonomy_2017Amended')
tax_db = pd.read_excel('Taxonomy_2017Amended.xlsm', 'Calculation Comp')
pres_db = pd.read_excel('Taxonomy_2017Amended.xlsm', 'Presentation Comp')

In [613]:
#retrieve only the sheet for a regular income statement for now
tax_db = tax_db.loc[tax_db['Definition Document'] == ' Statement of Income (Including Gross Margin)'].reset_index()
#retrieve only the sheet for a regular income statement for now
pres_db = pres_db.loc[pres_db['Definition Document'] == ' Statement of Income (Including Gross Margin)'].reset_index()

In [597]:
#join both databases, even stuff which is not found from the income statement
#on the XBRL taxonomy dictionary
frame = pd.merge(tax_db, test_fin, how = 'outer', on = 'tag')

In [591]:
#find the maximum depth of the analyzed statement
maxline = max(frame.dropna(subset = ['value']).depth)

#delete all the childs at higher depth then max depth
frame.drop(frame[(frame['depth'] >= maxline) & (frame['value'].isnull() == True)].index, inplace=True)

In [604]:
test_fin

,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,...,datp,dcml,report,line,stmt,inpth,rfile,prole,plabel,negating
0,0000909832-15-000014,SalesRevenueNet,us-gaap/2015,20150831,4,USD,0x00000000,0.0,1.136660e+11,None,...,1.0,-6.0,4,2,IS,0,H,terseLabel,Net sales,0
1,0000909832-15-000014,RevenueFromEnrollmentAndRegistrationFeesExclud...,us-gaap/2015,20150831,4,USD,0x00000000,0.0,2.533000e+09,None,...,1.0,-6.0,4,3,IS,0,H,terseLabel,Membership fees,0
2,0000909832-15-000014,Revenues,us-gaap/2015,20150831,4,USD,0x00000000,0.0,1.161990e+11,None,...,1.0,-6.0,4,4,IS,0,H,totalLabel,Total revenue,0
3,0000909832-15-000014,CostOfGoodsSold,us-gaap/2015,20150831,4,USD,0x00000000,0.0,1.010650e+11,None,...,1.0,-6.0,4,6,IS,0,H,terseLabel,Merchandise costs,0
4,0000909832-15-000014,SellingGeneralAndAdministrativeExpense,us-gaap/2015,20150831,4,USD,0x00000000,0.0,1.144500e+10,None,...,1.0,-6.0,4,7,IS,0,H,terseLabel,"Selling, general and administrative",0
5,0000909832-15-000014,PreOpeningCosts,us-gaap/2015,20150831,4,USD,0x00000000,0.0,6.500000e+07,None,...,1.0,-6.0,4,8,IS,0,H,terseLabel,Preopening expenses,0
6,0000909832-15-000014,OperatingIncomeLoss,us-gaap/2015,20150831,4,USD,0x00000000,0.0,3.624000e+09,None,...,1.0,-6.0,4,9,IS,0,H,totalLabel,Operating income,0
7,0000909832-15-000014,InterestExpense,us-gaap/2015,20150831,4,USD,0x00000000,0.0,1.240000e+08,None,...,1.0,-6.0,4,11,IS,0,H,negatedLabel,Interest expense,1
8,0000909832-15-000014,InterestAndOtherIncome,us-gaap/2015,20150831,4,USD,0x00000000,0.0,1.040000e+08,None,...,1.0,-6.0,4,12,IS,0,H,terseLabel,"Interest income and other, net",0
9,0000909832-15-000014,IncomeLossFromContinuingOperationsBeforeIncome...,us-gaap/2015,20150831,4,USD,0x00000000,0.0,3.604000e+09,None,...,1.0,-6.0,4,13,IS,0,H,totalLabel,INCOME BEFORE INCOME TAXES,0


In [606]:
pres_db

,index,Definition Number,Definition Type,Definition Document,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,prefix,tag,label,pres_depth,pres_order,priority,parent,arcrole,preferredLabel,systemid
0,1616,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,IncomeStatementAbstract,Income Statement [Abstract],0,NaN,NaN,NaN,NaN,NaN,NaN
1,1617,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,StatementTable,Statement [Table],1,10.0,0.0,us-gaap:IncomeStatementAbstract,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
2,1618,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,StatementScenarioAxis,Scenario [Axis],2,10.0,0.0,us-gaap:StatementTable,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
3,1619,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,ScenarioUnspecifiedDomain,"Scenario, Unspecified [Domain]",3,10.0,0.0,us-gaap:StatementScenarioAxis,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
4,1620,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,ScenarioPreviouslyReportedMember,"Scenario, Previously Reported [Member]",4,10.0,0.0,us-gaap:ScenarioUnspecifiedDomain,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
5,1621,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,RestatementAdjustmentMember,Restatement Adjustment [Member],4,20.0,0.0,us-gaap:ScenarioUnspecifiedDomain,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
6,1622,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,StatementLineItems,Statement [Line Items],2,20.0,0.0,us-gaap:StatementTable,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
7,1623,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,NetIncomeLossAvailableToCommonStockholdersBasi...,Net Income (Loss) Available to Common Stockhol...,3,10.0,0.0,us-gaap:StatementLineItems,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
8,1624,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,NetIncomeLossAbstract,Net Income (Loss) Attributable to Parent [Abst...,4,10.0,0.0,us-gaap:NetIncomeLossAvailableToCommonStockhol...,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml
9,1625,124100,Statement,Statement of Income,NaN,NaN,NaN,NaN,us-gaap,NetIncomeLossAttributableToReportingEntityAbst...,"Net Income (Loss), Including Portion Attributa...",5,10.0,0.0,us-gaap:NetIncomeLossAbstract,http://www.xbrl.org/2003/arcrole/parent-child,NaN,us-gaap-stm-soi-egm-pre-2017-01-31.xml


In [592]:
#calculates the values
for i in range(int(maxline),0,-1):
#i = 12
    #delete all the tags at that depth which do not have values
    frame.drop(frame[(frame['depth'] == i) & (frame['value'].isnull() == True)].index, inplace=True)
    #identify the frame at that depth
    dp_frame = frame.loc[frame.depth == i]
    for j in dp_frame.index:
        #find the location of the child
        chi_loc = dp_frame.loc[j]
        #erase the first 8 characters of the parent child to find the parent
        par_name = chi_loc['parent'].encode('utf-8')[8:]
        par_idx = frame.loc[frame.tag == par_name].index[0]
        #if the value was not filled in the XBRL instance document
        if pd.isnull(frame.loc[par_idx]['adsh']) == True:
            chi_val = dp_frame.loc[j]['value']
            chi_wei = dp_frame.loc[j]['weight']
            par_val = frame.loc[par_idx]['value']
            
            if pd.isnull(par_val) == True:
                par_val = 0
            #sum the value of the parent and the child
            par_val = par_val + (chi_val * chi_wei)
            #set the value of the parent as the value of the parent in the frame
            frame.set_value(par_idx, 'value', par_val)
#delete all the childs at depth 0 if they have no value
frame.drop(frame[(frame['depth'] == 0) & (frame['value'].isnull() == True)].index, inplace=True)

ValueError: cannot convert float NaN to integer

In [576]:
pres_frame = pd.merge(frame, pres_db, how = 'inner', on = 'tag')

In [580]:
pres_frame = pres_frame.drop_duplicates(subset = 'tag', keep = 'first', inplace=False).reset_index()

In [581]:
len(pres_frame)

27

In [584]:
pres_frame[['tag','pres_depth','pres_order','parent_x','value']].sort_values(['pres_depth','pres_order']).to_csv('testframe1.csv')

In [487]:
#frame.to_csv('testframe.csv')